## 用户流失数据预测 ##
    1.数字型数据-处理数据（去除不要的列，转换文本字符）
    2.对数据进行数值压缩（归一化，标准化，二值化）
    3.选择分类器（这里构造了很多可选的分类器）
    4.选择评估指标
    5.对预测值的处理

   ***
   **Q：**
   
    1.如何进行变量输出？
    2.对数据进行归一化，标准化（StandardScaler），二值化
    3.直接替换DataFrame里面的一列，法一：apply，法二：参考下面的表达式
    4.如何一次性去除很多没有用的列
    5.如何把文本型数据转成数值型数据
    6.如何构造多个分类器-函数形式-返回类别（下面有返回概率的）
    7.如何进行交叉验证
    8.如何写评估指标-函数形式
    9.如何构造多个分类器-函数形式-返回概率）

In [72]:
import numpy as np
import pandas as pd

   **1.如何进行变量输出**
    1..format()

In [73]:
churn_df = pd.read_csv('./churn.csv',encoding="utf-8")
print("Column names:{}".format(churn_df.columns.tolist()))
print(churn_df.head())
print(len(churn_df))

Column names:['State', 'Account Length', 'Area Code', 'Phone', "Int'l Plan", 'VMail Plan', 'VMail Message', 'Day Mins', 'Day Calls', 'Day Charge', 'Eve Mins', 'Eve Calls', 'Eve Charge', 'Night Mins', 'Night Calls', 'Night Charge', 'Intl Mins', 'Intl Calls', 'Intl Charge', 'CustServ Calls', 'Churn?']
  State  Account Length  Area Code     Phone Int'l Plan VMail Plan  \
0    KS             128        415  382-4657         no        yes   
1    OH             107        415  371-7191         no        yes   
2    NJ             137        415  358-1921         no         no   
3    OH              84        408  375-9999        yes         no   
4    OK              75        415  330-6626        yes         no   

   VMail Message  Day Mins  Day Calls  Day Charge   ...    Eve Calls  \
0             25     265.1        110       45.07   ...           99   
1             26     161.6        123       27.47   ...          103   
2              0     243.4        114       41.38   ...       

   **2.对数据进行归一化，标准化（StandardScaler），二值化**
    
   **3.直接替换DataFrame里面的一列，法一：apply，法二：参考下面的表达式**
    
   **4.如何一次性去除很多没有用的列**
   
   **5.如何把文本型数据转成数值型数据**

In [74]:
#数据处理
from sklearn.preprocessing import StandardScaler
import re

#1.对标签进行处理，这里单独拿出来了
y = np.where(churn_result == 'True.',1,0)

#2.去掉没用的列
to_drop = ['State','Area Code','Phone','Churn?']
churn_feat_space = churn_df.drop(to_drop,axis=1)

#3.把yes和no的值都转成0，1
yes_no = ["Int'l Plan",'VMail Plan']
churn_feat_space[yes_no] = churn_feat_space[yes_no] == 'yes'

#4.压缩数值
X = churn_feat_space.as_matrix().astype(np.float)
print(X)
scaler = StandardScaler()#
X = scaler.fit_transform(X)
print(X[0])
print(len(y[y == 0]))#流失
print(len(y[y == 1]))#不流失，不均衡现象

[[128.     0.     1.   ...   3.     2.7    1.  ]
 [107.     0.     1.   ...   3.     3.7    1.  ]
 [137.     0.     0.   ...   5.     3.29   0.  ]
 ...
 [ 28.     0.     0.   ...   6.     3.81   2.  ]
 [184.     1.     0.   ...  10.     1.35   2.  ]
 [ 74.     0.     1.   ...   4.     3.7    0.  ]]
[ 0.67648946 -0.32758048  1.6170861   1.23488274  1.56676695  0.47664315
  1.56703625 -0.07060962 -0.05594035 -0.07042665  0.86674322 -0.46549436
  0.86602851 -0.08500823 -0.60119509 -0.0856905  -0.42793202]
2850
483


**6.如何构造多个分类器-函数形式-返回类别（下面有返回概率的）**

**7.如何进行交叉验证**

In [75]:
#分类器集合
from sklearn.cross_validation import KFold

def run_cv(X,y,clf_class,**kwargs):
    #构造交叉对象
    kf = KFold(len(y),n_folds=5,shuffle=True)
    y_pred = y.copy() #下面的预测结果直接在上面改，牛
    
    #迭代每一次交叉验证
    for train_index, test_index in kf:
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        
        #用各种参数初始化一个分类器
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        y_pred[test_index] = clf.predict(X_test)
        return y_pred

**8.如何写评估指标-函数形式**

In [76]:
#评估指标
import numpy as np
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score 

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier as RF
from sklearn.neighbors import KNeighborsClassifier as KNN

def accuracy(y_true, y_pred):
    print("accuracy_score:{}".format(accuracy_score(y_true, y_pred)))
    print("precision_score:{}".format(precision_score(y_true, y_pred)))
    print("recall_score:{}".format(recall_score(y_true, y_pred)))
    #return np.mean(y_true == y_pred)#算术平均数，其实也算是准确率

print("Support vector machines:")
accuracy(y,run_cv(X,y,SVC))
print("Random forest:")
accuracy(y,run_cv(X,y,RF))
print("K-nearest-neighbors:")
accuracy(y,run_cv(X,y,KNN))

Support vector machines:
accuracy_score:0.9831983198319832
precision_score:0.988558352402746
recall_score:0.8944099378881988
Random forest:
accuracy_score:0.987998799879988
precision_score:0.9825708061002179
recall_score:0.9337474120082816
K-nearest-neighbors:
accuracy_score:0.9756975697569757
precision_score:0.9785714285714285
recall_score:0.8509316770186336


**9.如何构造多个分类器-函数形式-返回概率）**

    定流失用户的优先级，有的用户有60%的可能流失，有的有90%,先处理后者的情况
    把之前算的东西设置成概率值，规定一个可以容忍的值，多少概率才是流失！

In [77]:
def run_prob_cv(X,y,clf_class,**kwargs):
    kf = KFold(len(y),n_folds=5,shuffle=True)
    y_prob = np.zeros((len(y),2))
    for train_index, test_index in kf:
        X_train, X_test = X[train_index],X[test_index]
        y_train = y[train_index]
        clf = clf_class(**kwargs)
        clf.fit(X_train, y_train)
        #预测概率，而不是类别
        y_prob[test_index] = clf.predict_proba(X_test)#注意这个赋值方法
    return y_prob

pred_prob = run_prob_cv(X, y, RF, n_estimators=10)
print(pred_prob)

[[0.9 0.1]
 [0.9 0.1]
 [1.  0. ]
 ...
 [0.8 0.2]
 [0.7 0.3]
 [0.8 0.2]]


**下面是概率的例子，但是我没怎么看懂**

    之前看过一次，得到这个概率值后，去类别1的概率，对这个概率可以进行的操作是：加上一点值。然后在round()一下，去评估指标那里。

In [78]:
import warnings
warnings.filterwarnings('ignore')

pred_prob = run_prob_cv(X, y, RF, n_estimators=10)
print(pred_prob)
pred_churn = pred_prob[:,1]
is_churn = y ==1
print("test")
print(is_churn)

counts = pd.value_counts(pred_churn)
print("sd")
print(counts)

true_prob = {}
for prob in counts.index:
    true_prob[prob] = np.mean(is_churn[pred_churn == prob])
    true_prob = pd.Series(true_prob)
    
counts = pd.concat([counts,true_prob],axis=1).reset_index()
counts.columns = ['pred_prob','count','true_prob']
counts

[[1.  0. ]
 [1.  0. ]
 [1.  0. ]
 ...
 [0.9 0.1]
 [0.7 0.3]
 [1.  0. ]]
test
[False False False ... False False False]
sd
0.0    1739
0.1     734
0.2     253
0.3     143
0.9      81
0.7      75
0.8      72
0.6      63
0.5      61
0.4      56
1.0      56
dtype: int64


,pred_prob,count,true_prob
0,0.0,1739,0.028177
1,0.1,734,0.034060
2,0.2,253,0.027668
3,0.3,143,0.139860
4,0.9,81,0.987654
5,0.7,75,0.946667
6,0.8,72,0.916667
7,0.6,63,0.698413
8,0.5,61,0.721311
9,0.4,56,0.375000
